In [1]:
import pickle
# import PreProcessing functions
import sys 
import os
sys.path.append(os.path.abspath("C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\NLP-Classification\\nlp_project\\preProcessing"))
from fake_News_Classifier_preProcessing import Transformations

import pandas as pd
import numpy as np
# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Import required pipeline and transformation libraries
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Import processing and evaluation libraries
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Text Vectorizers
from sklearn.feature_extraction.text import CountVectorizer #for bag of words
from sklearn.feature_extraction.text import TfidfVectorizer #TF-IDF library

# Import Model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

# for scoring
from sklearn.metrics import fbeta_score, make_scorer

# Import features

In [2]:
#Import training features
import pathlib
path_to_read_model = '..\\probleam_study' #Path of current working Directory
with open(path_to_read_model + '\\trainFeatures_list.pkl', 'rb') as f:
    X, y = pickle.load(f)

# Save Column Names of feature Training DataSet

In [3]:
feature_ColumnNames = X.columns
import pathlib
path_to_write_output=str(pathlib.Path.cwd()) #Path of current working Directory
with open(path_to_write_output + '\\feature_ColumnNames.pkl', 'wb') as handle:
 pickle.dump(feature_ColumnNames, handle)

In [4]:
feature_ColumnNames

Index(['id', 'title', 'author', 'text'], dtype='object')

# Data PreProcessing, Transformation and Outlier Elimination

In [5]:
# Define a function to apply the imported functions
def preProcessing(featureDF, targetDF, function_list):
    for function in function_list:
        featureDF, targetDF = function(featureDF, targetDF)
    return featureDF, targetDF

X, y = preProcessing(X, y, [Transformations])

>>feature shape before cleaning:  (5824, 4)
>>target shape before cleaning:  (5824,)
>>feature shape after cleaning:  (5641, 1)
>>target shape after cleaning:  (5641,)
>>feature shape after preProcessing:  (5641, 1)
>>target shape after preProcessing:  (5641,)


# Check the feature columns

In [6]:
X.columns

Index(['title'], dtype='object')

In [7]:
X.isnull().sum()

title    0
dtype: int64

In [8]:
y.isnull().sum()

0

In [9]:
X.shape

(5641, 1)

In [10]:
y.shape

(5641,)

# Modelling

In [11]:
# Python Dictonary to store the Vectorizers and their parameters
#vectorizer_grid = [CountVectorizer(), TfidfVectorizer()]
# Python Dictonary to store the models and their parameters
#modell_grid ={
#    LogisticRegression(tol=0.0001, random_state=1):{
#        'model__penalty': ['l1', 'l2', 'elasticnet'],
#        'model__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
#        'model__max_iter': list(range(100,800,1500)),
#        'model__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
#        'textVec__ngram_range': [(1,1), (1, 2), (1,3)],
#        'textVec__max_features': [None, 1500, 15000]
#    },
#    RandomForestClassifier(random_state=1):{
#        'model__n_estimators':[10, 100, 500, 1000],
#        'model__criterion':['gini', 'entropy'],
#        'model__max_depth':[10, 50, 100, 1000, None],
#        'model__max_features': ['sqrt', 'log2'],
#        'textVec__ngram_range': [(1,1), (1, 2), (1,3)],
#        'textVec__max_features': [None, 1500, 15000]
#    },
#    MultinomialNB():{
#        'model__alpha': [0, 1, 2],
#        'model__fit_prior': [True, False],
#        'textVec__ngram_range': [(1,1), (1, 2), (1,3)],
#        'textVec__max_features': [None, 1500, 15000]        
#    },
#    GaussianNB():{
#        'model__var_smoothing': [1e-12, 1e-9, 1e-6, 1e-3],
#        'textVec__ngram_range': [(1,1), (1, 2), (1,3)] ,
#        'textVec__max_features': [None, 1500, 15000]       
#    },
#    BernoulliNB():{
#        'model__alpha': [0, 1, 2],
#        'model__fit_prior': [True, False],
#        'textVec__ngram_range': [(1,1), (1, 2), (1,3)],
#        'textVec__max_features': [None, 1500, 15000]        
#    }
#}
vectorizer_grid = [CountVectorizer(), TfidfVectorizer()]
modell_grid ={
    BernoulliNB():{
        'model__alpha': [0, 1, 2],
        'model__fit_prior': [True, False],
        'textVec__ngram_range': [(1,1), (1, 2), (1,3)],
        'textVec__max_features': [None, 1500, 15000, 30000]        
    },
    MultinomialNB():{
        'model__alpha': [0, 1, 2],
        'model__fit_prior': [True, False],
        'textVec__ngram_range': [(1,1), (1, 2), (1,3)],
        'textVec__max_features': [None, 1500, 15000, 30000]        
    }
}

## Theory for scoring  
	False negatives acceptable=> minimize False positives=>(optimize for prescision or Specificity) ex: spam filter => beta<1 for F-Beta score.  
	False positives acceptable=> minimize False negatives=>(optimize for Recall or Sensitivity) ex: Fradualent transactions => beta>1 for F-Beta score.  

In [12]:
#Empty DataFrame to store the results from the CrossValidation Matrix
full_df = pd.DataFrame()
best_algos = {}

#Iterate and fit the above specified Model and parameter dictonary
for curr_Vectorizer in vectorizer_grid:
    for curr_model, model_params in modell_grid.items():
            print(">>Model: ",curr_model)
            print("Current vectorizer: ",curr_Vectorizer)
            print("parameters: ",model_params)
            print('\n')
            
            # Building the Scaler for distance and gradient descent based algorithms
            algo_name = str(curr_model).split('(')[0]
            textVec_name = str(curr_Vectorizer).split('(')[0]
            curr_scaler = MinMaxScaler((0,1)) if algo_name == 'LogisticRegression' or algo_name == 'KNeighborsClassifier'  else None
            
            # Buiding Modelling Pipe line
            modelling_pipe = Pipeline([
                ('textVec', curr_Vectorizer),
                ('scaler', curr_scaler),
                ('model', curr_model)
            ])
            
            ##GridSearch K folds cross validation definition with current model and its parameters
            ### scoring='accuracy' for balanced datasets
            ### scoring= 'balanced_accuracy' or 'f1' for unbalanced datasets
            ### False negatives acceptable=>(optimize for prescision or Specificity) ex: spam filter
            # cv_curr =  GridSearchCV(modelling_pipe, model_params, cv=5, return_train_score=True, scoring='balanced_accuracy', verbose=3, n_jobs=-1)
            #cv_curr =  RandomizedSearchCV(modelling_pipe, model_params, cv=5, return_train_score=True, scoring='accuracy', verbose=3, n_jobs=-1, n_iter=100)
            ## to tune for precision and Recall using custom scorer via F-Beta Score
            ### use average='macro' for mutilable target classification in the scoring method
            cv_curr =  GridSearchCV(modelling_pipe, model_params, cv=5, 
                                    return_train_score=True, 
                                    scoring=make_scorer(fbeta_score, beta=0.75, average='macro', greater_is_better=True), 
                                    verbose=3, n_jobs=-1)
            ##fit the data to the defined grid search
            display(cv_curr)
            cv_curr.fit(X.title, y)
                        
            ##Create a DataFrame out of the CrossValidation results
            all_res = pd.DataFrame(cv_curr.cv_results_)
            
            ##Create a temp Datframe with only values of 'params', 'mean_test_score' from CrossValidation results
            temp = all_res.loc[:, ['params', 'mean_test_score']]
            
            ##Get the name of the Model in use from the specified Model and parameter dictonary
            temp['algo'] = algo_name+'_'+textVec_name
            
            ##Merge the temporary dataframes and results to final DataFrame and Dictonary
            full_df = pd.concat([full_df, temp])
            best_algos[algo_name+'_'+textVec_name]={}
            best_algos[algo_name+'_'+textVec_name]['best_estimator'] = cv_curr.best_estimator_
            best_algos[algo_name+'_'+textVec_name]['best_mean_test_score'] = cv_curr.best_score_
            print('-'*50)

>>Model:  BernoulliNB()
Current vectorizer:  CountVectorizer()
parameters:  {'model__alpha': [0, 1, 2], 'model__fit_prior': [True, False], 'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)], 'textVec__max_features': [None, 1500, 15000, 30000]}




GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('textVec', CountVectorizer()),
                                       ('scaler', None),
                                       ('model', BernoulliNB())]),
             n_jobs=-1,
             param_grid={'model__alpha': [0, 1, 2],
                         'model__fit_prior': [True, False],
                         'textVec__max_features': [None, 1500, 15000, 30000],
                         'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             return_train_score=True,
             scoring=make_scorer(fbeta_score, beta=0.75, average=weighted),
             verbose=3)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
--------------------------------------------------
>>Model:  MultinomialNB()
Current vectorizer:  CountVectorizer()
parameters:  {'model__alpha': [0, 1, 2], 'model__fit_prior': [True, False], 'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)], 'textVec__max_features': [None, 1500, 15000, 30000]}




GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('textVec', CountVectorizer()),
                                       ('scaler', None),
                                       ('model', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'model__alpha': [0, 1, 2],
                         'model__fit_prior': [True, False],
                         'textVec__max_features': [None, 1500, 15000, 30000],
                         'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             return_train_score=True,
             scoring=make_scorer(fbeta_score, beta=0.75, average=weighted),
             verbose=3)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
--------------------------------------------------
>>Model:  BernoulliNB()
Current vectorizer:  TfidfVectorizer()
parameters:  {'model__alpha': [0, 1, 2], 'model__fit_prior': [True, False], 'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)], 'textVec__max_features': [None, 1500, 15000, 30000]}




GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('textVec', TfidfVectorizer()),
                                       ('scaler', None),
                                       ('model', BernoulliNB())]),
             n_jobs=-1,
             param_grid={'model__alpha': [0, 1, 2],
                         'model__fit_prior': [True, False],
                         'textVec__max_features': [None, 1500, 15000, 30000],
                         'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             return_train_score=True,
             scoring=make_scorer(fbeta_score, beta=0.75, average=weighted),
             verbose=3)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
--------------------------------------------------
>>Model:  MultinomialNB()
Current vectorizer:  TfidfVectorizer()
parameters:  {'model__alpha': [0, 1, 2], 'model__fit_prior': [True, False], 'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)], 'textVec__max_features': [None, 1500, 15000, 30000]}




GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('textVec', TfidfVectorizer()),
                                       ('scaler', None),
                                       ('model', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'model__alpha': [0, 1, 2],
                         'model__fit_prior': [True, False],
                         'textVec__max_features': [None, 1500, 15000, 30000],
                         'textVec__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             return_train_score=True,
             scoring=make_scorer(fbeta_score, beta=0.75, average=weighted),
             verbose=3)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
--------------------------------------------------


In [13]:
#View the best Regressor Models per each Algorithm and thier scores on the Training Data
best_algos

{'BernoulliNB_CountVectorizer': {'best_estimator': Pipeline(steps=[('textVec', CountVectorizer(ngram_range=(1, 2))),
                  ('scaler', None),
                  ('model', BernoulliNB(alpha=2, fit_prior=False))]),
  'best_mean_test_score': 0.9170658580863235},
 'MultinomialNB_CountVectorizer': {'best_estimator': Pipeline(steps=[('textVec',
                   CountVectorizer(max_features=15000, ngram_range=(1, 3))),
                  ('scaler', None),
                  ('model', MultinomialNB(alpha=1, fit_prior=False))]),
  'best_mean_test_score': 0.8861378107583524},
 'BernoulliNB_TfidfVectorizer': {'best_estimator': Pipeline(steps=[('textVec', TfidfVectorizer(ngram_range=(1, 2))),
                  ('scaler', None),
                  ('model', BernoulliNB(alpha=2, fit_prior=False))]),
  'best_mean_test_score': 0.9170658580863235},
 'MultinomialNB_TfidfVectorizer': {'best_estimator': Pipeline(steps=[('textVec',
                   TfidfVectorizer(max_features=15000, ngram_range

In [14]:
# Check for the best Regressor Model
full_df.sort_values('mean_test_score', ascending=False)

,params,mean_test_score,algo
61,"{'model__alpha': 2, 'model__fit_prior': False, 'textVec__max_features': None, 'textVec__ngram_range': (1, 2)}",0.917066,BernoulliNB_CountVectorizer
61,"{'model__alpha': 2, 'model__fit_prior': False, 'textVec__max_features': None, 'textVec__ngram_range': (1, 2)}",0.917066,BernoulliNB_TfidfVectorizer
49,"{'model__alpha': 2, 'model__fit_prior': True, 'textVec__max_features': None, 'textVec__ngram_range': (1, 2)}",0.916669,BernoulliNB_CountVectorizer
49,"{'model__alpha': 2, 'model__fit_prior': True, 'textVec__max_features': None, 'textVec__ngram_range': (1, 2)}",0.916669,BernoulliNB_TfidfVectorizer
58,"{'model__alpha': 2, 'model__fit_prior': True, 'textVec__max_features': 30000, 'textVec__ngram_range': (1, 2)}",0.916493,BernoulliNB_TfidfVectorizer
...,...,...,...
12,"{'model__alpha': 0, 'model__fit_prior': False, 'textVec__max_features': None, 'textVec__ngram_range': (1, 1)}",0.760835,MultinomialNB_TfidfVectorizer
18,"{'model__alpha': 0, 'model__fit_prior': False, 'textVec__max_features': 15000, 'textVec__ngram_range': (1, 1)}",0.760835,MultinomialNB_TfidfVectorizer
6,"{'model__alpha': 0, 'model__fit_prior': True, 'textVec__max_features': 15000, 'textVec__ngram_range': (1, 1)}",0.757782,MultinomialNB_TfidfVectorizer
9,"{'model__alpha': 0, 'model__fit_prior': True, 'textVec__max_features': 30000, 'textVec__ngram_range': (1, 1)}",0.757782,MultinomialNB_TfidfVectorizer


In [15]:
# Check for the best parameters and its score
print(">> best estimator: ",best_algos['BernoulliNB_TfidfVectorizer']['best_estimator'])
print(">> best_score: ",best_algos['BernoulliNB_TfidfVectorizer']['best_mean_test_score'])

>> best estimator:  Pipeline(steps=[('textVec', TfidfVectorizer(ngram_range=(1, 2))),
                ('scaler', None),
                ('model', BernoulliNB(alpha=2, fit_prior=False))])
>> best_score:  0.9170658580863235


# Saving the best estimators

In [16]:
estimators = [best_algos]
import pathlib
path_to_write_output=str(pathlib.Path.cwd()) #Path of current working Directory
with open(path_to_write_output + '\\estimators.pkl', 'wb') as handle:
    pickle.dump(estimators, handle)